# Keypoints generation with YOLO11

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path

from random import sample

import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
    model = YOLO("/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/YOLO/runs/pose/train17/weights/best.pt")

In [ ]:
!mkdir "/content/YOLO_INFERENCE/"
!mkdir "/content/labels/"
!mkdir "/content/low_confidance/"
!mkdir "/content/low_confidance/labels/"

In [ ]:
source_images = "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/ORATE singole 2"
target_folder = "/content/YOLO_INFERENCE/"
labels_folder = "/content/labels/"
low_confidance_folder = "/content/low_confidance/"
low_confidance_folder_labels = "/content/low_confidance/labels/"
source_images = list(Path(source_images).glob("*.jpg"))
sample_source_images = sample(source_images, 100)

### only 0 and 1 lens are used for keypoints detection as the segmentation will detect the corresponding mask in lens 3 and 2 respectilvely

In [ ]:
source_images_0 = [x for x in sample_source_images if "l0" in str(x)]
source_images_1= [x for x in sample_source_images if "l1" in str(x)]

In [ ]:
results = model.predict(sample_source_images)


0: 1024x800 7 fishs, 123.9ms
1: 1024x800 9 fishs, 123.9ms
2: 1024x800 3 fishs, 123.9ms
3: 1024x800 9 fishs, 123.9ms
4: 1024x800 4 fishs, 123.9ms
5: 1024x800 6 fishs, 123.9ms
6: 1024x800 13 fishs, 123.9ms
7: 1024x800 4 fishs, 123.9ms
8: 1024x800 1 fish, 123.9ms
9: 1024x800 5 fishs, 123.9ms
10: 1024x800 4 fishs, 123.9ms
11: 1024x800 9 fishs, 123.9ms
12: 1024x800 4 fishs, 123.9ms
13: 1024x800 3 fishs, 123.9ms
14: 1024x800 5 fishs, 123.9ms
15: 1024x800 3 fishs, 123.9ms
16: 1024x800 9 fishs, 123.9ms
17: 1024x800 6 fishs, 123.9ms
18: 1024x800 10 fishs, 123.9ms
19: 1024x800 3 fishs, 123.9ms
20: 1024x800 5 fishs, 123.9ms
21: 1024x800 4 fishs, 123.9ms
22: 1024x800 8 fishs, 123.9ms
23: 1024x800 8 fishs, 123.9ms
24: 1024x800 7 fishs, 123.9ms
25: 1024x800 5 fishs, 123.9ms
26: 1024x800 5 fishs, 123.9ms
27: 1024x800 (no detections), 123.9ms
28: 1024x800 4 fishs, 123.9ms
29: 1024x800 4 fishs, 123.9ms
30: 1024x800 4 fishs, 123.9ms
31: 1024x800 5 fishs, 123.9ms
32: 1024x800 4 fishs, 123.9ms
33: 1024x8

In [ ]:
def generate_labels(results, target_folder, labels_folder , confidence=.2):
    ""
    for result in results:
        image_file = f"{target_folder}{Path(result.path).stem}.jpg"
        result.save(image_file , line_width = 8 ,kpt_radius = 10 )
        labels_file = f"{labels_folder}{Path(result.path).stem}.txt"
        height, width, _ = cv2.imread(image_file).shape
        with open(labels_file , 'w') as label_file:
            for box, keypoint, conf in zip(result.boxes.xyxy, result.keypoints.xy, result.boxes.conf ):
                if conf >= confidence:
                #save them in yolo format
                    box_width = box[2] - box[0]
                    box_height = box[3] - box[1]
                    box_center_x = box[0] + box_width / 2
                    box_center_y = box[1] + box_height / 2

                    # Normalize the box coordinates
                    box_center_x /= width
                    box_center_y /= height
                    box_width /= width
                    box_height /= height

                    box_str = f"0 {box_center_x} {box_center_y} {box_width} {box_height}"

                    keypoints_str = " ".join([f"{x / width} {y / height}" for x, y in keypoint])

                    label_file.write(f"{box_str} {keypoints_str}\n")

In [ ]:
def generate_labels(results, target_folder, labels_folder, confidence: float = 0.2, line_width: int = 8, kpt_radius: int = 10) -> None:
    """
    Generate YOLO-style label files with bounding boxes and keypoints from detection results.

    Each detection result is saved as:
      - An annotated image in `target_folder`.
      - A `.txt` file in `labels_folder` containing YOLO-format annotations:
        class_id center_x center_y width height kpt1_x kpt1_y ... kptN_x kptN_y

    Bounding boxes and keypoints are normalized to [0, 1] relative to image size.

    Args:
        results (list): List of detection results, each containing `.path`, `.boxes.xyxy`, `.boxes.conf`, and `.keypoints.xy`.
        target_folder (str or Path): Directory where annotated images will be saved.
        labels_folder (str or Path): Directory where YOLO-format label files will be saved.
        confidence (float, optional): Minimum confidence threshold for saving a detection. Defaults to 0.2.
        line_width (int, optional): Line thickness for saved bounding box annotations. Defaults to 8.
        kpt_radius (int, optional): Radius for keypoint drawing. Defaults to 10.

    Returns:
        None
    """
    target_folder = Path(target_folder)
    labels_folder = Path(labels_folder)
    target_folder.mkdir(parents=True, exist_ok=True)
    labels_folder.mkdir(parents=True, exist_ok=True)

    for result in results:
        image_file = target_folder / f"{Path(result.path).stem}.jpg"
        labels_file = labels_folder / f"{Path(result.path).stem}.txt"

        # Save annotated image
        result.save(str(image_file), line_width=line_width, kpt_radius=kpt_radius)

        # Load image size
        img = cv2.imread(str(image_file))
        if img is None:
            print(f"[WARN] Could not read {image_file}, skipping.")
            continue
        height, width, _ = img.shape

        with open(labels_file, "w") as f:
            for box, keypoint, conf in zip(result.boxes.xyxy, result.keypoints.xy, result.boxes.conf):
                if conf < confidence:
                    continue

                # Bounding box in YOLO format (normalized)
                box_width = (box[2] - box[0]) / width
                box_height = (box[3] - box[1]) / height
                box_center_x = (box[0] + (box[2] - box[0]) / 2) / width
                box_center_y = (box[1] + (box[3] - box[1]) / 2) / height

                # Format bounding box
                box_str = f"0 {box_center_x:.6f} {box_center_y:.6f} {box_width:.6f} {box_height:.6f}"

                # Format keypoints (normalized)
                keypoints_str = " ".join([f"{x / width:.6f} {y / height:.6f}" for x, y in keypoint])

                f.write(f"{box_str} {keypoints_str}\n")


In [ ]:
generate_labels(results, target_folder = "./YOLO_INFERENCE/", labels_folder="./labels/")

In [ ]:
# copy the labels folder to /content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/YOLO...
import shutil
shutil.copytree(labels_folder, "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/YOLO_INFERENCE20/")

'/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/YOLO_INFERENCE20/'

# Inference for Validation

In [ ]:
!mkdir "/content/YOLO_INFERENCE/"
!mkdir "/content/labels/"
!mkdir "/content/low_confidance/"
!mkdir "/content/low_confidance/labels/"

In [ ]:
source_images_0 = "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/ORATE (Capraia)_Luglio 25"
source_images_1 = "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/SPIGOLE (Capraia)_Ottobre 24 con dati biometrici pesca"
source_images_2 = "/content/drive/MyDrive/2023_24 Prototype 2.0/sagome PESCI/Sagome in acqua"
source_images_3 = "/content/drive/MyDrive/2023_24 Prototype 2.0/sagome PESCI/Sagome a terra distanza nota"



### split images into lenses

In [ ]:
source_image = list(Path(source_images_0).glob("*.png"))

In [ ]:
!mkdir fish_images
!mkdir "/content/YOLO_INFERENCE/"
!mkdir "/content/labels/"

In [ ]:


for i , f in enumerate(source_image):
    fname = str(f)
    img = cv2.imread(fname)
    height, width = img.shape[:2]
    l0 = img[:height // 2, :width // 2]
    l3 = img[:height // 2, width // 2:]
    l1 = img[height // 2:, :width // 2]
    l2 = img[height // 2:, width // 2:]
    correct_l0 = cv2.rotate(l0, cv2.ROTATE_90_CLOCKWISE)
    correct_l1 = cv2.rotate(l1, cv2.ROTATE_90_CLOCKWISE)
    correct_l2 = cv2.rotate(l2, cv2.ROTATE_90_CLOCKWISE)
    correct_l3 = cv2.rotate(l3, cv2.ROTATE_90_CLOCKWISE)
    cv2.imwrite(f"./fish_images/{f.stem}_l0.jpg", correct_l0)
    cv2.imwrite(f"./fish_images/{f.stem}_l1.jpg", correct_l1)
    cv2.imwrite(f"./fish_images/{f.stem}_l2.jpg", correct_l2)
    cv2.imwrite(f"./fish_images/{f.stem}_l3.jpg", correct_l3)


In [ ]:
fish = Path('fish_images/').glob('*.*')
# only select the 0 and 1 lenses
fish = [x for x in fish if "l0" in str(x) or "l1" in str(x)]

In [ ]:
len(fish)

324

In [ ]:
# clear gpu ram
#import torch
#torch.cuda.empty_cache()

# Define a batch size that fits into your GPU memory
batch_size = 32 # You might need to adjust this value based on your GPU

# Process images in batches
results = []
for i in tqdm(range(0, len(fish), batch_size)):
    batch = fish[i:i + batch_size]
    batch_results = model.predict(batch)
    results.extend(batch_results)


In [ ]:
generate_labels(results, target_folder = "./YOLO_INFERENCE/", labels_folder="./labels/")

In [ ]:
target_folder = "/content/ORATE25/"
labels_folder = "/content/labels/"
low_confidance_folder = "/content/low_confidance/"
low_confidance_folder_labels = "/content/low_confidance/labels/"


In [ ]:
!zip -r /content/ORATE25.zip /content/orate25

# Spigole

In [ ]:
!mkdir fish_images
!mkdir "/content/YOLO_INFERENCE/"
!mkdir "/content/labels/"

In [ ]:
source_image = list(Path(source_images_1).glob("*/*.*"))

In [ ]:
for i , f in enumerate(source_image):
    try :
        fname = str(f)
        img = cv2.imread(fname)
        height, width = img.shape[:2]
        l0 = img[:height // 2, :width // 2]
        l3 = img[:height // 2, width // 2:]
        l1 = img[height // 2:, :width // 2]
        l2 = img[height // 2:, width // 2:]
        correct_l0 = cv2.rotate(l0, cv2.ROTATE_90_CLOCKWISE)
        correct_l1 = cv2.rotate(l1, cv2.ROTATE_90_CLOCKWISE)
        correct_l2 = cv2.rotate(l2, cv2.ROTATE_90_CLOCKWISE)
        correct_l3 = cv2.rotate(l3, cv2.ROTATE_90_CLOCKWISE)
        cv2.imwrite(f"./fish_images/{f.stem}_l0.jpg", correct_l0)
        cv2.imwrite(f"./fish_images/{f.stem}_l1.jpg", correct_l1)
        cv2.imwrite(f"./fish_images/{f.stem}_l2.jpg", correct_l2)
        cv2.imwrite(f"./fish_images/{f.stem}_l3.jpg", correct_l3)
    except :
        print(f)

/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/SPIGOLE (Capraia)_Ottobre 24 con dati biometrici pesca/Gabbia7_shut8_foc16_iso300_600_nuvoloso_12.00/Spigole capraia_23-10-24.xlsx
/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/SPIGOLE (Capraia)_Ottobre 24 con dati biometrici pesca/Gabbia7_shut8_foc16_iso300_600_nuvoloso_12.00/composizione Gabbia 7 (ottobre2024) Spigole.xlsx
/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/SPIGOLE (Capraia)_Ottobre 24 con dati biometrici pesca/Gabbia7_shut8_foc16_iso300_600_nuvoloso_12.00/distribuzione pesi Gabbia 7 (ottobre2024) Spigole.xlsx
/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/SPIGOLE (Capraia)_Ottobre 24 con dati biometrici pesca/Gabbia8_shut8_foc16_iso300_600_nuvoloso_14.00/distribuzione pesi startup israeliana.xlsx


In [ ]:
fish = Path('fish_images/').glob('*.*')
# only select the 0 and 1 lenses
fish = [x for x in fish if "l0" in str(x) or "l1" in str(x)]

In [ ]:
len(fish)

424

In [ ]:
# clear gpu ram
#import torch
#torch.cuda.empty_cache()

# Define a batch size that fits into your GPU memory
batch_size = 32 # You might need to adjust this value based on your GPU

# Process images in batches
results = []
for i in tqdm(range(0, len(fish), batch_size)):
    batch = fish[i:i + batch_size]
    batch_results = model.predict(batch)
    results.extend(batch_results)

# save to drive

In [ ]:
generate_labels(results, target_folder = "./YOLO_INFERENCE/", labels_folder="./labels/")

In [ ]:
# copy the labels folder to /content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/YOLO...
import shutil
shutil.copytree("/content/labels_spigole24", "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/spigole24/")

'/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/spigole24/'

In [ ]:
# copy the labels folder to /content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/YOLO...
import shutil
shutil.copytree("/content/labels_orate25", "/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/orate25/")

'/content/drive/MyDrive/2023_24 Prototype 2.0/PESCI/Keypoints/Data_Inference/orate25/'

In [ ]:
!zip -r /content/spigole24.zip /content/spigole24_predictions/